# Consolidating Route Patterns in GTFS

Occurs after running R script that filters the trips, schedules, etc. by a chosen date.
This part first came after an import to TransCAD but this is an alternate approach of consolidating solely based on GTFS instead of trying to join the calculated trips per Route_Name back to the route_pattern_id. There isn't a great field to join back on as we don't know exactly how TransCAD is translating the GTFS information into the Route_Names and how the routes are being consolidated as the number of unique route patterns differs between TransCAD and GTFS with GTFS having less unique route patterns in its trip table *(both post R filtering)

As discussed with Marty and Sabiheh, the goal of this consolidation step is to re-assign route patterns with low numbers of trips across the day ( less than three in each time period) to route_pattern_ids with the most trips within their matching Route, Direction, and Headsign. This gets filtered down to the trip level where if a trip's route pattern is consolidated, it will be replaced with the route_pattern_id with max trips.

Notes:
- All trips with same headsign/route/direction  are assigned the same route_pattern_id. The route_pattern_id that represents the group is the route_pattern_id with the most trips per day.
- If less than 3 trips per tod period across all day, assign the route_pattern_id of the dir/route group with the most trips.

- The MBTA service day is 3AM - 26:59. For TOD assignment, everything that isn't between the hours of 6:30 - 19:00 is counted as NT. This addresses this issue as it includes 19:00-26:39 AND 3:00 - 6:30.


Needs/Steps:
- Number of trips per time period per route_pattern_id
    - Midpoint time of each trip
    - Each trip classified by TOD (based on midpoint)
    - Sum of trips per TOD by route_pattern_id
- route_pattern_id with most daily trips per Route, Direction, Headsign
    - Sum all tod trips per route_pattern_id
    - grab just the max per Route, Direction, Headsign (but keep route_pattern_id)
- consolidate route patterns by Route, Direction, Headsign
    - if route_pattern_id has less than 3 trips in each of the 4 TODs, replace with max trips route_pattern_id

In [1]:
import time
import datetime

import matplotlib
matplotlib.use('agg')  # allows notebook to be tested in Travis

import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import pandana as pdna
import time

import urbanaccess as ua
from urbanaccess.config import settings
from urbanaccess.gtfsfeeds import feeds
from urbanaccess import gtfsfeeds
from urbanaccess.gtfs.gtfsfeeds_dataframe import gtfsfeeds_dfs
from urbanaccess.network import ua_network, load_network

from tqdm import tqdm

%matplotlib inline

In [2]:
# required bbox including all of Massachusetts and RI as well as parts of NH, CT, NY
bbox = (-73.7207, 41.1198, -69.7876, 43.1161)
# path to the downloaded and cleaned gtfs - mbta recap file for fall 2018
#   this could also be a folder of gtfs folders (pre merge of multiple gtfs)

path_to_gtfs = r"J:\Shared drives\TMD_TSA\Model\networks\Transit\gtfs\mbta_2019\Part_2_GTFS_R\mbta2018_102418_20221207" #r"J:\Shared drives\TMD_TSA\Model\networks\Transit\gtfs\bnrd\1_gtfs_r"
out_path = r"J:\Shared drives\TMD_TSA\Model\networks\Transit\gtfs\mbta_2019\Part_3_Py_Consolidation\mbta2018_post_py2" #r"J:\Shared drives\TMD_TSA\Model\networks\Transit\gtfs\bnrd\2_gtfs_py"

In [3]:
loaded_feeds = ua.gtfs.load.gtfsfeed_to_df(gtfsfeed_path= path_to_gtfs,
                                           validation=True,
                                           verbose=True,
                                           bbox=bbox,
                                           remove_stops_outsidebbox=False,
                                           append_definitions=True)

Checking GTFS text file header whitespace... Reading files using encoding: utf-8 set in configuration.
GTFS text file header whitespace check completed. Took 0.66 seconds
--------------------------------
Processing GTFS feed: mbta2018_102418_20221207
The unique agency id: mbta was generated using the name of the agency in the agency.txt file.
Unique agency id operation complete. Took 0.01 seconds
Unique GTFS feed id operation complete. Took 0.00 seconds
No GTFS feed stops were found to be outside the bounding box coordinates
mbta2018_102418_20221207 GTFS feed stops: coordinates are in northwest hemisphere. Latitude = North (90); Longitude = West (-90).
Appended route type to stops
Appended route type to stop_times
--------------------------------
Added descriptive definitions to stops, routes, stop_times, and trips tables
Successfully converted ['departure_time'] to seconds past midnight and appended new columns to stop_times. Took 1.08 seconds
1 GTFS feed file(s) successfully read as 

#### Separate Out Bus Routes

In [ ]:
# separate out the bus routes and non-bus routes if multiple mode types
def separate_bus(route_table,trip_table):
    non_bus_routes = list(route_table.query('route_type != 3').route_id.unique())
    non_bus_trips = trip_table.query('route_id in @non_bus_routes')
    bus_trips = trip_table.query('route_id not in @non_bus_routes')

    if len(non_bus_trips) == 0:
        print("All routes in this GTFS are bus routes.")
    if len(bus_trips) == 0:
        print("There are no bus routes in this GTFS.")
    return(non_bus_trips, bus_trips)

In [ ]:
non_bus_trips, bus_trips = separate_bus(gtfsfeeds_dfs.routes, gtfsfeeds_dfs.trips)

#### Get Trips per TOD (per route_pattern_id)

In [ ]:
def get_start_stop_times(stop_times):    
    '''for every trip, grab the start time and stop time of the trip'''
    chocula =0 
    for trip_id in stop_times['trip_id'].unique():
        max_row = stop_times.query('trip_id==@trip_id').query('stop_sequence == stop_sequence.max()')[['trip_id','arrival_time']]
        min_row = stop_times.query('trip_id==@trip_id').query('stop_sequence == stop_sequence.min()')[['trip_id','arrival_time']]
        r2 = min_row.merge(max_row, how='left', on='trip_id', suffixes = ('_start','_end'))
        if chocula == 0:
            flintstone = pd.DataFrame(r2)
        else:
            flintstone=pd.concat([flintstone,r2])
        chocula +=1
    return(flintstone)


In [ ]:
def assign_tod(start_stop):
    '''calculate midpoint of trip, use midpoint to assign TOD'''
    
    start_stop['at_end_dec'] = (
        (
            (start_stop['arrival_time_end'].str.split(":").str[0]).astype('int32')
            +
            ((start_stop['arrival_time_end'].str.split(":").str[1]
            ).astype('int32')/60)))
    start_stop['at_start_dec'] = (
        (
            (start_stop['arrival_time_start'].str.split(":").str[0]).astype('int32')
            +
            ((start_stop['arrival_time_start'].str.split(":").str[1]
            ).astype('int32')/60)))
    
    start_stop['midpoint'] = start_stop['at_start_dec'] + ((start_stop['at_end_dec']-start_stop['at_start_dec'])/2)
    start_stop['tod'] = np.where(start_stop['midpoint'].between(6.50,9.50),'AM', np.where(
        start_stop['midpoint'].between(9.50,15.00), 'MD', np.where(
            start_stop['midpoint'].between(15.00,19.00),'PM', 'NT' 
        )
            ) 
        )
    
    return start_stop


In [ ]:
start_stop = get_start_stop_times(gtfsfeeds_dfs.stop_times) # simpson

In [ ]:
start_stop_tod = assign_tod(start_stop) # smurf

#### Determine route_pattern_id with Max Number of Daily Trips
- Getting duplicate ids where same number of trips for max, so choosing one arbitrarily.
- The section can be expanded to try to determine which has the most trips during peak period, but for now this works.

In [ ]:
# FOR R/H/D, select RPID with most trips
def max_daily_trips(trips):
        # add TOD to trip table
    tod_trips = trips.merge(start_stop_tod[['trip_id','tod']], how='left', on='trip_id')

        # get the number of trips per R/H/D & rpid
    day_rpid = tod_trips.groupby(by=['route_id','trip_headsign','direction_id','route_pattern_id']).agg({'trip_id':'nunique'})
    day_rpid = day_rpid.rename(columns = {'trip_id':'daily_trips'}).reset_index()

        # for each R/H/D, select just the route_pattern_id with the most daily trips
    max_rpid = day_rpid.groupby(by=['route_id','trip_headsign','direction_id']).apply(lambda g: g[g['daily_trips'] == g['daily_trips'].max()])[['route_pattern_id','daily_trips']].reset_index()
    max_rpid = max_rpid[['route_id','trip_headsign','direction_id','route_pattern_id']].rename(columns = {'route_pattern_id':'route_pattern_id_new'})

    # because there are several cases where the max is held by two different route_pattern_ids, chose one arbitrarily
    max_rpid = max_rpid.drop_duplicates(subset=['route_id','trip_headsign','direction_id'])
    return(max_rpid)

In [ ]:
max_trips_rpid = max_daily_trips(bus_trips)

In [ ]:
# update rpid to rpid with max trips
def update_rpid_max(trips, max_trips_rpid, base_trip_columns):
    # join trip table with the RPID with most trips (join is on R/H/D, RPID is an attribute of the table)
    trips_update_rpid = trips.merge(max_trips_rpid, how='left', on=['route_id','trip_headsign','direction_id'])

    # update the route_pattern_ids based on R/H/D max
        # this may be the same as original RPID as this should cover all R/H/D combos (therefore all trips)
    trips_update_rpid['route_pattern_id'] = trips_update_rpid['route_pattern_id_new']
    trips_update_rpid = trips_update_rpid[base_trip_columns]
    
    return(trips_update_rpid)

In [ ]:
trips_update1 = update_rpid_max(bus_trips, max_trips_rpid, gtfsfeeds_dfs.trips.columns)

#### Update route/headsign/direction combos where each TOD period has less than three trips per route_pattern_id
- take the route_pattern_id with max number of daily trips per route/direction

In [ ]:
def update_RHD_under_3(trips_update_rpid, start_stop_tod):
    # add tod back to trips table
    tod_trips = trips_update_rpid.merge(start_stop_tod[['trip_id','tod']], how='left', on='trip_id')

    # for each TOD and R/H/D count trips per TOD, get number of unique RPIDs, and unique ServiceIDs
    tod_rpid = tod_trips.groupby(by=['route_id','trip_headsign','direction_id','tod']).agg({'tod':'count','route_pattern_id':'nunique','service_id':'nunique'})
    tod_rpid = tod_rpid.rename(columns = {'tod':'trips_per_tod'}).reset_index()

    # select just the trips that need to be updated
    tod_rpid_4update = tod_rpid[['route_id','trip_headsign','direction_id','tod','trips_per_tod']]
        # for R/H/D, get trips_per_tod separated into columns by TOD
    tod_rpid_pivot = tod_rpid_4update.pivot_table(index = ['route_id','trip_headsign','direction_id'], columns = ['tod'])

        # get just the trips where all 4 periods have less than 3 trips
    needs_update = tod_rpid_pivot['trips_per_tod'].reset_index().fillna(0).query('AM < 3 & MD < 3 & PM < 3 & NT < 3')

    return(needs_update)

In [ ]:
trips_need_update = update_RHD_under_3(trips_update1, start_stop_tod)

In [ ]:
# get the number of trips per R/D & rpid
day_rpid2 = tod_trips2.groupby(by=['route_id','direction_id','route_pattern_id']).agg({'trip_id':'nunique'})
day_rpid2 = day_rpid2.rename(columns = {'trip_id':'daily_trips'}).reset_index()

    # for each R/D, select just the route_pattern_id with the most daily trips
max_rpid2 = day_rpid2.groupby(by=['route_id','direction_id']).apply(lambda g: g[g['daily_trips'] == g['daily_trips'].max()])[['route_pattern_id','daily_trips']].reset_index()
max_rpid2 = max_rpid2[['route_id','direction_id','route_pattern_id']].rename(columns = {'route_pattern_id':'route_pattern_id_new'})

# because there are several cases where the max is held by two different route_pattern_ids, chose one arbitrarily
max_rpid2 = max_rpid2.drop_duplicates(subset=['route_id','direction_id'])

max_rpid2

In [ ]:
# update the route_pattern_ids for just R/H/D combos that have < 3 trips in each of the 4 TOD periods
update_table = needs_update.merge(max_rpid2, how='left', on=['route_id','direction_id'])

update_table = update_table.drop_duplicates(subset=['route_id','direction_id','trip_headsign'])

update_table

#### Update those route_pattern_ids!

In [ ]:
trips_update3 = trips_update1.merge(
    update_table[['route_id','direction_id','trip_headsign','route_pattern_id_new']], 
    how='left',on=['route_id','direction_id','trip_headsign'])

trips_update3['route_pattern_id'] = np.where(
    (
        (trips_update3['route_pattern_id_new'].isna())), 
    trips_update3['route_pattern_id'], 
    trips_update3['route_pattern_id_new'])

In [ ]:
# update headsign for all trips with updated route_pattern_id
rpid_th = trips_update3.groupby(by=['route_pattern_id','trip_headsign','block_id','shape_id','service_id']).agg({'trip_id':'count'}).reset_index()
rpid_th_max = rpid_th.groupby(by=['route_pattern_id']).apply(lambda g: g[g['trip_id'] == g['trip_id'].max()]).reset_index(drop=True)
rpid_th_max = rpid_th_max.drop_duplicates(subset='route_pattern_id')
trips_update4 = trips_update3.merge(
    rpid_th_max[['route_pattern_id','trip_headsign','block_id','shape_id','service_id']], 
    how='left',on=['route_pattern_id'], suffixes=(None, '_to_rplce'))
trips_update4['trip_headsign'] = trips_update4['trip_headsign_to_rplce']
trips_update4['block_id'] = trips_update4['block_id_to_rplce']
#trips_update4['shape_id'] = trips_update4['shape_id_to_rplce']
trips_update4['service_id'] = trips_update4['service_id_to_rplce']
trips_update4

#### CHECKS

In [ ]:
trips_update3[trips_update3['trip_id'].duplicated(keep=False)]

len(trips_update4['trip_id'].unique())
len(trips_update4['route_pattern_id'].unique())
len(bus_trips['route_pattern_id'].unique())


In [ ]:
trips_update5 = trips_update4[gtfsfeeds_dfs.trips.columns].merge(bus_trips[['trip_id','route_pattern_id','trip_headsign']], how='left', on='trip_id', suffixes=(None,'_old'))

In [ ]:
len(trips_update5.query('route_pattern_id != route_pattern_id_old').query('trip_headsign == trip_headsign_old'))


#### Replace Trips Table

In [ ]:
trips_keep_safe = gtfsfeeds_dfs.trips
# keep trips just busses for next part too
gtfsfeeds_dfs.trips = trips_update5

### Create Generic Stop Times & Stop Sequence per Route Pattern

- Can't just change route_pattern_id as TransCAD does not use this field to combine trips into routes. There is no effect on the import.
- Working theory is that to consolidate routes one must use the stop_times.txt table as it defines the stop sequence for every trip. Theoretically, this is being used to consolidate trips into routes based on whether they have the same stop sequence.

Plan:
- Explore if stop times differ depending on TOD or if only realtime GTFS takes into account traffic.
    - For every route_pattern_id, get the average trip length (in minutes).
- For every route_pattern_id, get the average number of minutes between each pair of stops in the stop sequence.
- Replace the stop times and sequence for trips that had their route_pattern_id updated with the generic stop sequence and times per route_pattern_id created in the previous step. 
    - Keep the start time and work off of that.
    - Arrival time will equal departure time given that the difference is usually less than a minute. Will assume difference in time can be included in the minutes to next arrival time for aggregate modeling purposes.

In [ ]:
def generic_stop_times_sequence(stop_times, trips):
    # flag the first stop in every trip
    stop_times['first_stop'] =  0
    stop_times.loc[stop_times.groupby('trip_id').stop_sequence.idxmin(),'first_stop'] = 1

    rpid_stop_dict = {}
    # for each route_pattern_id calculate the most common stop pattern
    for rpid in tqdm(trips['route_pattern_id'].unique()):
        patterns = []
        rpid_trips = trips.query('route_pattern_id == @rpid')['trip_id'].to_list()
        rpid_stops = stop_times.query('@rpid_trips in trip_id').sort_values('stop_sequence')
        
        # for every trip, get stop pattern and save into a list of lists
        for tid in rpid_stops['trip_id']:
            patterns.append([tid,':'.join(rpid_stops.query('trip_id == @tid')['stop_id'].to_list())])
        
        # make data frame of every trip and its pattern of stops
        df = pd.DataFrame(patterns, columns = ['trip_id','stop_pattern'])
        # get most common stop_pattern
        df_grby = df.groupby('stop_pattern').count().reset_index()
        max_stop_pattern = df_grby.query('trip_id == trip_id.max()')['stop_pattern'].to_list()
        # get all the trip_ids that represent the route_pattern_id's most popular stop pattern
        rep_trips = df.query('stop_pattern == @max_stop_pattern[0]')['trip_id'].to_list()

        rpid_stop_dict[rpid] = rep_trips

    return(rpid_stop_dict)

In [ ]:
rpid_stop_dict = generic_stop_times_sequence(gtfsfeeds_dfs.stop_times, gtfsfeeds_dfs.trips)

In [ ]:
def update_to_majority_stop_pattern(trips, rpid_stop_dict, stop_times):
    # for every row where the route_pattern_id has been updated & doesn't have the majority stop pattern
    trip_stop_replace = {}
    trip_shape_replace = {}
    for idx, row in tqdm(gtfsfeeds_dfs.trips.iterrows()):
        # identify trip_id
        tid = row['trip_id']
        # get the route_pattern_id for the trip
        rpid = row['route_pattern_id']

        if (tid not in rpid_stop_dict[rpid]): # if trip_id is not in the majority stop pattern
            # get all of the trips associated with that route_pattern_id (and stop sequence pattern)
            all_trips = rpid_stop_dict[rpid]

            # get the start time (for first stop) for trip_id
            start_time = stop_times.query('(trip_id == @tid) & (first_stop == 1)')
            # get the start time (for first stop) for all trips that share same route_pattern_id
            all_start_times = stop_times.query('(trip_id in @all_trips) & (first_stop == 1)')

            # create list getting the difference in start times between the selected trip and all the other trips that share the same route_pattern_id
            test_list = [[x,(abs(start_time['departure_time_sec']- all_start_times.query('trip_id == @x'))['departure_time_sec'].iloc[0])] for x in all_start_times['trip_id']]
            close = {}
            close = {sub[0]:sub[1] for sub in test_list}
                
            # get trip with the minimum difference in start time    
            min_t = min(close, key=close.get)
            if tid != min_t:
                trip_stop_replace[tid] = min_t
                trip_shape_replace[tid] = trips.query('trip_id == @min_t')['shape_id'].values[0]
        else:
            trip_shape_replace[tid] = trips.query('trip_id == @tid')['shape_id'].values[0]
    
    trip_shape_replace_tab = pd.DataFrame.from_dict(
        trip_shape_replace, orient='index').reset_index().rename(
            columns = {'index':'trip_id', 0:'shape_id_update'})
    
    trips['shape_id_new'] = trips['trip_id'].apply(lambda x: trip_shape_replace[x])

    trips['shape_id'] = trips['shape_id_new']

    # keep only relevant columns
    trips_clean = trips.drop(columns=['route_pattern_id_old','trip_headsign_old','shape_id_new'])

    return(trips_clean, trip_stop_replace)

In [ ]:
trips, trip_stop_replace = update_to_majority_stop_pattern(gtfsfeeds_dfs.trips, rpid_stop_dict, gtfsfeeds_dfs.stop_times)

In [ ]:
def update_stop_times_new_pattern(stop_times, trips, trip_stop_replace, stop_times_columns):
    stop_times = stop_times.sort_values('stop_sequence')
    # for every trip (in dictionary with value = trip sharing route_pattern_id with closest start time)
    for tid in tqdm(trip_stop_replace.values()):
    # update value of time_between_stops for every stop in the selected trip
        # time between stops calculated by subtracting the prior departure_time_sec from current (why stop_sequence order is important)
        stop_times.loc[stop_times.loc[:,'trip_id'] == str(tid), 'time_between_stops'] = stop_times.loc[stop_times.loc[:,'trip_id'] == str(tid), 'departure_time_sec'].diff()

    for trip in tqdm(trip_stop_replace.keys()):
        start_time = stop_times.query('(trip_id == @trip) & (first_stop == 1)')['departure_time_sec']
        # drop old stop times
        stop_times = stop_times.drop(
            stop_times.loc[stop_times['trip_id']==trip].index)
        # grab new stop times
        new_trip = trip_stop_replace[trip]
        nst = stop_times.query('trip_id == @new_trip')
        nst['trip_id'] = trip

        # replace the start time, then calculate the stop times by the departure_time_sec difference
        nst.loc[nst.loc[:,'first_stop']==1,'departure_time_sec'] = int(start_time.iloc[0])
        nst.loc[nst.loc[:,'first_stop']==1,'time_between_stops'] = int(start_time.iloc[0])
        nst['departure_time_sec'] = nst['time_between_stops'].cumsum()

        # recalc arrival/dep times
        nst['arrival_time'] = pd.to_datetime(nst['departure_time_sec'],unit='s').astype('str').str[11:19]
        nst['departure_time'] = nst['arrival_time']

        #keep only relevant columns
        nst = nst[stop_times.columns]

        stop_times = pd.concat([stop_times,nst])
    # keep only relevant columns and sort
    stop_times = stop_times[stop_times_columns].sort_values(by=['trip_id','stop_sequence'])
    return(stop_times)

In [ ]:
stop_times = update_stop_times_new_pattern(gtfsfeeds_dfs.stop_times, gtfsfeeds_dfs.trips, trip_stop_replace, gtfsfeeds_dfs.stop_times.columns)
gtfsfeeds_dfs.stop_times = stop_times

In [ ]:
# merge back the filtered out trips
gtfsfeeds_dfs.trips = pd.concat([trips, non_bus_trips])

In [ ]:
gtfsfeeds_dfs.stop_times.to_csv(out_path+r"/stop_times.txt")
gtfsfeeds_dfs.trips.to_csv(out_path+r"/trips.txt")